<a href="https://colab.research.google.com/github/yellayujin/mulcamp34/blob/main/240111_Statistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 0110: ADsP pptx로 강의 진행

# 문제 상황(p.294)
- 평균 무게 130g으로 알려진 감튀-> 1개 구매, 무게 122.02g
- 14개 표본 평균 계산-> 128.451g

- Q. 어떤 통계 검정 방법을 사용해야 할까?
    : 정규성 검정 후 일표본 T검정
    
- Q. 귀무가설과 대립가설을 설정하시오
    H0: 평균은 130g, H1: 평균은 130g이 아님 (= 양측검정)
    H0: 평균은 130g, H1: 평균은 130g보다 작다 (= 단측검정)


In [ ]:
import numpy as np
import pandas as pd
import scipy
from scipy import stats
scipy.__version__

'1.10.1'

In [ ]:
%precision 3
np.random.seed(1111)

In [ ]:
df = pd.read_csv('data/ch11_potato.csv')
df.head()

,무게
0,122.02
1,131.73
2,130.60
3,131.82
4,132.05


In [ ]:
sample = np.array(df['무게'])     # 변수 하나밖에 없어도 df로 하면 다르게 됨(각각에 [ ] 들어감)
sample                           # 배열로 해야하는 이유가 있을까?

array([122.02, 131.73, 130.6 , 131.82, 132.05, 126.12, 124.43, 132.89,
       122.79, 129.95, 126.14, 134.45, 127.64, 125.68])

In [ ]:
# 표본평균
s_mean = np.mean(sample)   # array 방식 평균
s_mean

128.451

In [ ]:
s_mean1 = sample.mean()
s_mean1

128.451

# A학생의 관심(단측 검정)
- 14개 표본평균이 모평균 130g보다 작은지 여부
- 여기서는 감자튀김의 모집단이 정규분포를 따르고 있다고 가정(~N(130, 9)으로 모평균, 모분산 기지) -> 정규성 검정 할 필요 없음
    + 비모수검정 쓸 필요 없음 -> 모수 검정 하면 됨

In [ ]:
rv = stats.norm(130, np.sqrt(9/14))  # 표본평균이 따르는 정규분포
rv.isf(0.95)   # P(Xbar<=x) = 0.05인 x인 임계값(신뢰구간 구할 수 있음)

128.681

## 해석
- 표본평균이 128.681이하일 확률은 0.05임
- A학생이 추출한 표본평균이 128.451이었던 것은 매우 드물게 발생하는 사건이었음
- 하지만 여전히 샘플 수가 작아 확신하기는 힘듬(yet, 의심의 수준)
    + 샘플의 수가 바뀔 때마다 임계값의 변화가 있음
    + Thus, t-통계량(=검정통계량)과 p-값 확인   (검정통계량: 각 검정방법에 따른 통계량)

# Way1(유의수준 이용)

## 검정통계량 구하기


In [ ]:
z = (s_mean - 130) / np.sqrt(9/14)
z

-1.932

## (검정통계량에 대한)임계값 구하기

In [ ]:
rv = stats.norm()    # 표준정규분포에 대한 임계값
rv.isf(0.95)         # 생존함수의 역함수(inverse survival function)

-1.645

## 해석
- z값이 기각역에 있으므로 귀무가설 기각
- 이 결론은 검정통계량으로 표본평균을 사용했을 때와 동일한 결론
- 여전히

# Way2(p-value이용)

## 누적분포함수 이용

In [ ]:
rv.cdf(z)

0.027

## 해석
- p-값이 유의수준 0.05보다 작으므로 귀무가설 기각

# 양측 검정

In [ ]:
z = (s_mean - 130) / np.sqrt(9/14)  # 표준화확률변수
rv = stats.norm()   # 표준정규분포
rv.interval(0.95)   # 95% 신뢰구간, 위 구간 안이면 채택역, 양 쪽 바깥이면 기각역

(-1.960, 1.960)

In [ ]:
rv.cdf(z) * 2       # p-value

0.053

## 해석
- p-value가 유의수준 0.05보다 크므로 귀무가설 기각

# 가설검정의 두 가지 오류
- 제1종 오류(H0이 옳은데 기각)
- 제2종 오류(H1이 옳은데 H0 채택)


## 제1종 오류
- 오탐(false positive)

In [ ]:
# 제1종 오류
rv = stats.norm(130, 3)
c = stats.norm().isf(0.95)
n_samples = 10000
cnt = 0

for _ in range(n_samples):
    sample_ = np.round(rv.rvs(14), 2)
    s_mean_ = np.mean(sample_)
    z = (s_mean_ - 130) / np.sqrt(9/14)
    if z < c:
        cnt += 1
cnt / n_samples

0.051

## 해석
- 제1종 오류를 범할 확률(유의수준, 위험률, 알파)는 위와 같음(돌릴 때마다 다르게 나옴)
- 좀 더 엄격하게 적용하고 싶다면 유의수준 1%에서 가설검정 수행

## 제2종 오류
- 실제 표본평균이 130g보다 작음을 확인했음(H1이 옳음)에도 불구하고 표본평균이 130g보다 작다라고 얻을 수 없는 상황(계산 오류 등)
    + 본래 검출해야하는 것을 검출하지 못했으므로 미탐(false negative)

In [ ]:
rv = stats.norm(128, 3)   # p.306
c = stats.norm().isf(0.95)
n_samples = 10000
cnt = 0
for _ in range(n_samples):
    sample_ = np.round(rv.rvs(14), 2)
    s_mean_ = np.mean(sample_)
    z = (s_mean_ - 130) / np.sqrt(9/14)
    if z >= c:
        cnt += 1

cnt / n_samples

0.196

## 해석
- 위 수치 -> 베타
- 검정력 = 1-베타이므로 제대로 탐지할 수 있는 능력은 약 80%
+ 제2종 오류는 언제나 모집단의 정보에 의존... 제어 불능
    + 위 상황에서는 모집단에 대한 정보를 안다고 가정(특수 상황)하여 계산 가능했음
+ 제1종 오류와 제2종 오류는 서로 상충관계...

# 모평균과 모분산의 미/기지 여부 따른 가설 검정
- 현실적으로 모평균과 모분산 아는 경우 거의 없음

# 정규분포의 모평균에 대한 검정: 모분산을 모르는 경우
- 모분산을 알지 못하는 상황에서 정규분포의 모평균에 대한 검정을 일표본 t-검정이라 함
- 검정통계량 t = (Xbar-모평균) / np.sqrt(표본분산/ 표본 수)

In [ ]:
def pmean_test(sample, mean0, alpha=0.05):
    s_mean = np.mean(sample)
    u_var = np.var(sample, ddof=1)
    n = len(sample)
    rv = stats.t(df=n-1)
    interval = rv.interval(1-alpha)

    t = (s_mean - mean0) / np.sqrt(u_var/n)
    print(f't통계량 값은 {t:.3f}')
    if interval[0] <= t <= interval[1]:
        print('귀무가설을 채택')
    else:
        print('귀무가설을 기각')

    if t < 0:
        p = rv.cdf(t) * 2   # 양측검정이므로 누적밀도함수의 값 2배
    else:
        p = (1 - rv.cdf(t)) * 2
    print(f'p값은 {p:.3f}')

In [ ]:
pmean_test(sample, 130)

t통계량 값은 -1.455
귀무가설을 채택
p값은 0.169


In [ ]:
from scipy.stats import ttest_1samp
t, p = stats.ttest_1samp(sample, 130)
t, p

(-1.455, 0.169)

# z-통계량과 t-통계량
- 모평균과 모분산 기지: z-검정, 이 검정에서 나오는 통계량이 z-통계량(but 현실에서 잘 없음)
- 표본을 추출하여 표본평균과 표본분산을 아는 상태에서 모집단에 대한 추측을 할 때: t-검정, 이 검정에서 나오는 통계량이 t-통계량


# 이표본 문제에 관한 가설검정
- 독립표본
    + 정규성 검정 통과: 독립표본 t-검정
    + 정규성 검정 불통: 맨-위트니의 U검정
- 대응표본
    + 정규성 검정 통과: 대응표본 t-검정
    + 정규성 검정 불통: 비모수 검정, 윌콕슨의 부호순위검정(: 중앙값의 차이에 대한 검정)

## 독립표본 t-검정
- p.318
- H0: 두 학급의 집중력의 평균 차이가 없다. (뮤a = 뮤b, 차이=0)
- H1: 두 학급의 집중력의 평균 차이가 있다. (뮤a != 뮤b, 차이!=0)

- 가정: 정규성(shapiro), 등분산성(levene)

In [ ]:
df0 = pd.read_csv('data/ch11_training_ind.csv')
df0.head()

,A,B
0,47,49
1,50,52
2,37,54
3,60,48
4,39,51


### 스튜던트의 방법, 웰치의 방법
- 스튜던트의 방법(ttest_ind의디폴트) 사용하기 위한 조건
1. 자료는 모두 동일 간격을 가진 연속형 수치
2. 두 집단은 서로 독립
3. 자료의 수치는 정규성 띰
4. 두 집단 각각에서 추정된 분산이 같다(등분산성 만족)

- 4.를 만족하지 못하면 웰치의 방법 사용(등분산성 만족X)


In [ ]:
# 웰치의 T-검정
t, p = stats.ttest_ind(df0['A'], df0['B'], equal_var=False)   # equal_var=False: 웰치의 방법, 두 집단 각각에서 추정된 분산이 다르다
t, p

(-1.761, 0.087)

In [ ]:
# 스튜던트의 T-검정
t, p = stats.ttest_ind(df0['A'], df0['B'], equal_var=True) # 두 집단 각각에서 추정된 분산이 같다
t, p

(-1.761, 0.086)

In [ ]:
# 귀무가설 채택(두 그룹의 평균 차이 없음)

## 대응표본 t-검정
- 특별한 제재(intervention) 가했을 때의 평균값 차이에 대한 검정

In [ ]:
import pandas as pd

df1 = pd.read_csv('data/ch11_training_rel.csv')
df1.head()

,전,후
0,59,41
1,52,63
2,55,68
3,61,59
4,59,84


In [ ]:
# H0: 운동 전후 집중력 평균의 차이가 없을 것이다(전=후, 차이=0)
# H1: 운동 전후 집중력 평균의 차이가 있을 것이다(전!=후, 차이!=0)

In [ ]:
df1['차'] = df1['후']-df1['전']
df1.head()

,전,후,차
0,59,41,-18
1,52,63,11
2,55,68,13
3,61,59,-2
4,59,84,25


In [ ]:
t, p = stats.ttest_1samp(df1['차'], 0)   # 얘는 차(diff)를 따로 계산해 줬어야. but 아래는 따로 할 필요 없음
t, p

(2.204, 0.040)

In [ ]:
# 유의수준 5%에서 임계값은 +-1.96, 채택역은 (-1.96, 1.96), 바깥은 기각역
# 기각역 안에 검정통계량이 놓이므로 귀무가설 기각, 차이가 있다고 볼 수 있음

In [ ]:
t, p = stats.ttest_rel(df1['후'], df1['전'])     # 이 방법으로 숙지해두자
t, p

(2.204, 0.040)

# 비모수검정
- 모수 검정에서 각 가정을 만족시키지 못할 때 사용

## 윌콕슨의 부호순위 검정
- 대응표본에서 정규성 검정 통과 못 할 때
    + 평균차이가 아닌 중앙값 차이에 대한 검정임!

In [ ]:
df2 = pd.read_csv('data/ch11_training_rel.csv')
df3 = df2[:6].copy()
df3['차'] = df3['후']-df3['전']
df3

,전,후,차
0,59,41,-18
1,52,63,11
2,55,68,13
3,61,59,-2
4,59,84,25
5,45,37,-8


In [ ]:
# 순위 적용
diff = df3['차']
rank = stats.rankdata(abs(diff)).astype(int)
df3['순위'] = rank
df3

,전,후,차,순위
0,59,41,-18,5
1,52,63,11,3
2,55,68,13,4
3,61,59,-2,1
4,59,84,25,6
5,45,37,-8,2


In [ ]:
# 차의 부호가 +인 것의 순위합, 차의 부호가 -인 것의 순위합을 구해야
r_minus = np.sum((diff<0)*rank)
r_plus = np.sum((diff>0)*rank)

r_minus, r_plus    # 이 중 작은 쪽이 검정통계량(=8) 됨!
# 윌콕슨의 부호순위검정에서는 이 검정통계량이 임계값보다 작은 경우에 귀무가설 기각되는 단측검정 수행함
# 차이에 편차가 있으면 검정통계량이 작아짐

(8, 13)

In [ ]:
T, p = stats.wilcoxon(df3['전'], df3['후'])
T, p      # 귀무가설 기각됨(대응표본 t-검정과 동일한 결론)

(8.000, 0.688)

In [ ]:
# + 윌콕슨의 부호순위검정은 모집단이 정규분포를 따를 때도 사용 가능하나, 검정력이 낮음

# 적합도 검정(goodness-of-fit): by 카이제곱검정
- 어떤 실험에서 관측도수가 가정하는 이론상의 분포 또는 주어진 형태를 잘 따른다는 귀무가설을 검정하는 것
    - e.g.1 이미 분포가 주어진 상태(정규분포 등)에서 이 데이터가 실제로 그 분포를 잘 따르고 있는지 확인(정규분포에 대한 적합도 검정)
    - e.g.2 이미 등분산성이 만족된다고 알려졌을 때 실제로 등분산인지(등분산성에 대한 적합도 검정)
- 관측도수: 실제 실험에서 단일 특성에 의해 분류된 각 범주의 관측값(숫자, 갯수)
- 독립성 검정의 카이제곱검정과 워딩 차이 확인

In [ ]:
# 카이제곱분포의 상위 5%점(자유도5): x^2_0.05(5)
rv = stats.chi2(5)
rv.isf(0.05)

11.070

# 독립성 검정(교차분석): by 카이제곱검정
- 방법론이 동일하기에, 적합도검정을 위한 카이제곱검정인지 독립성검정을 위한 카이제곱검정인지 구분해야
    - 전자의 키워드: 분포, 등분산, 실제로 따름
    - 후자의 키워드: 두 범주형 변수, 독립, 상관(=한 특성이 다른 특성에 영향을 미치는지)
         e.g. 영화 장르별 간식 구매의 상관성
             - H0: 영화 장르와 간식류 구입은 서로 독립적
             - H1: 영화 장르와 간식류 구입은 서로 독립적인 것은 아니다(=상관성이 존재)
         + 두 연속형 변수이면 상관분석! This chisquare-test is only for categorical variables.
        
- 독립성 검정에서의 검정통계량의 자유도는 n-1이 아니라 (r-1)(c-1)로, 다름
- 이원 분할표를 통한 범주별 도수 파악
    - 이원분할표 = pd.crosstab(객체['변수1'], 객체['변수2'], margins = False/True(주변 도수 포함 여부)
    - chi2, p, dof, expected = chi2_contingency(이원분할표)